In [1]:
from pushcart.metadata import Metadata
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("escape", '"')
    .load("./dataset.csv")
)

print(df.schema)
df.show()

23/08/03 23:33:12 WARN Utils: Your hostname, laptop resolves to a loopback address: 127.0.1.1; using 192.168.2.13 instead (on interface enp0s20f0u1u4u4)
23/08/03 23:33:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/03 23:33:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


StructType([StructField('id', StringType(), True), StructField('ts', StringType(), True), StructField('payload', StringType(), True)])
+---+--------------------+--------------------+
| id|                  ts|             payload|
+---+--------------------+--------------------+
|  0|2023-07-13T17:26:...|{"current_page": ...|
|  1|2023-07-13T17:27:...|{"current_page": ...|
|  2|2023-07-13T17:27:...|{"current_page": ...|
|  3|2023-07-13T17:27:...|{"current_page": ...|
|  4|2023-07-13T17:27:...|{"current_page": ...|
|  5|2023-07-13T17:27:...|{"current_page": ...|
|  6|2023-07-13T17:27:...|{"current_page": ...|
|  7|2023-07-13T17:27:...|{"current_page": ...|
|  8|2023-07-13T17:27:...|{"current_page": ...|
+---+--------------------+--------------------+



In [3]:
md = Metadata(df, infer_fraction=1.0)
md.get_metadata()


2023-08-03 23:33:16.327 | INFO     | pushcart.metadata:_infer_timestamps:71 - Attempting to infer timestamp format for id column.
2023-08-03 23:33:16.536 | WARNING  | pushcart.metadata:_infer_timestamps:83 - Could not infer timestamp format for id column.
2023-08-03 23:33:16.537 | INFO     | pushcart.metadata:_infer_json_schema:27 - Attempting to infer JSON schema for id column.
2023-08-03 23:33:17.364 | WARNING  | pushcart.metadata:_infer_json_schema:33 - Could not infer JSON schema for id column.
2023-08-03 23:33:17.367 | INFO     | pushcart.metadata:_infer_timestamps:71 - Attempting to infer timestamp format for ts column.
2023-08-03 23:33:17.511 | INFO     | pushcart.metadata:_infer_timestamps:71 - Attempting to infer timestamp format for payload column.
2023-08-03 23:33:17.632 | WARNING  | pushcart.metadata:_infer_timestamps:83 - Could not infer timestamp format for payload column.
2023-08-03 23:33:17.633 | INFO     | pushcart.metadata:_infer_json_schema:27 - Attempting to infer J

In [4]:
import pandas as pd

data = md.metadata_grid.get_visible_data()
new_row = pd.DataFrame(
    columns=data.columns,
    index=[data.index[-1] + 1],
    data="" * len(data.columns),
)
data2 = pd.concat([data, new_row]).reset_index(drop=True)
# md.metadata_grid.data = data
display(data2)


,column_order,source_column_name,source_column_type,dest_column_name,dest_column_type,transform_function,default_value,validation_rule,validation_action
0,0,id,string,id,string,,,,
1,1,ts,string,ts,string,"F.to_timestamp(F.col(""ts""), ""yyyy-MM-dd'T'HH:m...",,,
2,2,payload,string,payload,string,"F.from_json(F.col(""payload""), schema=""struct<c...",,,
3,,,,,,,,,


In [5]:
# Edits are reflected in the underlying dataset
md.metadata_df


,column_order,source_column_name,source_column_type,dest_column_name,dest_column_type,transform_function,default_value,validation_rule,validation_action
0,0,id,string,id,string,,,,
1,1,ts,string,ts,string,"F.to_timestamp(F.col(""ts""), ""yyyy-MM-dd'T'HH:m...",,,
2,2,payload,string,payload,string,"F.from_json(F.col(""payload""), schema=""struct<c...",,,


In [6]:
code = md.generate_code()


2023-08-03 23:33:18.294 | INFO     | pushcart.metadata:_drop_technical_cols:408 - Excluding technical columns: None
2023-08-03 23:33:18.298 | INFO     | pushcart.metadata:generate_code:484 - 
df = (df
	.withColumn("ts", F.to_timestamp(F.col("ts"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSS"))
	.withColumn("payload", F.from_json(F.col("payload"), schema="struct<current_page:bigint,data:array<struct<fact:string,length:bigint>>,first_page_url:string,from:bigint,last_page:bigint,last_page_url:string,links:array<struct<active:boolean,label:string,url:string>>,next_page_url:string,path:string,per_page:bigint,prev_page_url:string,to:bigint,total:bigint>"))
	.select(['id', 'ts', 'payload']))


In [7]:
import pyspark.sql.functions as F

processed_df = (
    df.withColumn("id", F.col("id").cast("int"))
    .withColumn("ts", F.to_timestamp(F.col("ts"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSS"))
    .withColumn(
        "payload",
        F.from_json(
            F.col("payload"),
            schema="struct<current_page:bigint,data:array<struct<fact:string,length:bigint>>,first_page_url:string,from:bigint,last_page:bigint,last_page_url:string,links:array<struct<active:boolean,label:string,url:string>>,next_page_url:string,path:string,per_page:bigint,prev_page_url:string,to:bigint,total:bigint>",
        ),
    )
    .select(["id", "ts", "payload"])
)

print(processed_df.schema)
processed_df.show()

StructType([StructField('id', IntegerType(), True), StructField('ts', TimestampType(), True), StructField('payload', StructType([StructField('current_page', LongType(), True), StructField('data', ArrayType(StructType([StructField('fact', StringType(), True), StructField('length', LongType(), True)]), True), True), StructField('first_page_url', StringType(), True), StructField('from', LongType(), True), StructField('last_page', LongType(), True), StructField('last_page_url', StringType(), True), StructField('links', ArrayType(StructType([StructField('active', BooleanType(), True), StructField('label', StringType(), True), StructField('url', StringType(), True)]), True), True), StructField('next_page_url', StringType(), True), StructField('path', StringType(), True), StructField('per_page', LongType(), True), StructField('prev_page_url', StringType(), True), StructField('to', LongType(), True), StructField('total', LongType(), True)]), True)])
+---+--------------------+------------------

In [8]:
nested_md = Metadata(processed_df, infer_fraction=1.0)
nested_md.get_metadata()


2023-08-03 23:33:19.204 | INFO     | pushcart.metadata:_infer_timestamps:71 - Attempting to infer timestamp format for payload.data.fact column.
2023-08-03 23:33:19.479 | WARNING  | pushcart.metadata:_infer_timestamps:83 - Could not infer timestamp format for payload.data.fact column.
2023-08-03 23:33:19.480 | INFO     | pushcart.metadata:_infer_json_schema:27 - Attempting to infer JSON schema for payload.data.fact column.
2023-08-03 23:33:19.854 | WARNING  | pushcart.metadata:_infer_json_schema:33 - Could not infer JSON schema for payload.data.fact column.
2023-08-03 23:33:19.866 | INFO     | pushcart.metadata:_infer_timestamps:71 - Attempting to infer timestamp format for payload.first_page_url column.
2023-08-03 23:33:20.173 | WARNING  | pushcart.metadata:_infer_timestamps:83 - Could not infer timestamp format for payload.first_page_url column.
2023-08-03 23:33:20.175 | INFO     | pushcart.metadata:_infer_json_schema:27 - Attempting to infer JSON schema for payload.first_page_url co

In [9]:
flat_code = nested_md.generate_code()


2023-08-03 23:33:23.821 | INFO     | pushcart.metadata:_drop_technical_cols:408 - Excluding technical columns: None
2023-08-03 23:33:23.827 | INFO     | pushcart.metadata:generate_code:484 - 
df = (df
	.withColumn("payload_current_page", F.col("payload.current_page").cast("bigint"))
	.withColumn("payload_data", F.explode("payload.data"))
	.withColumn("payload_data_fact", F.col("payload_data.fact").cast("string"))
	.withColumn("payload_data_length", F.col("payload_data.length").cast("bigint"))
	.withColumn("payload_first_page_url", F.col("payload.first_page_url").cast("string"))
	.withColumn("payload_from", F.col("payload.from").cast("bigint"))
	.withColumn("payload_last_page", F.col("payload.last_page").cast("bigint"))
	.withColumn("payload_last_page_url", F.col("payload.last_page_url").cast("string"))
	.withColumn("payload_links", F.explode("payload.links"))
	.withColumn("payload_links_active", F.col("payload_links.active").cast("boolean"))
	.withColumn("payload_links_label", F.col("p

In [10]:
flattened_df = (
    processed_df.withColumn(
        "payload_current_page", F.col("payload.current_page").cast("bigint")
    )
    .withColumn("payload_data", F.explode("payload.data"))
    .withColumn("payload_data_fact", F.col("payload_data.fact").cast("string"))
    .withColumn("payload_data_length", F.col("payload_data.length").cast("bigint"))
    .withColumn(
        "payload_first_page_url", F.col("payload.first_page_url").cast("string")
    )
    .withColumn("payload_from", F.col("payload.from").cast("bigint"))
    .withColumn("payload_last_page", F.col("payload.last_page").cast("bigint"))
    .withColumn("payload_last_page_url", F.col("payload.last_page_url").cast("string"))
    .withColumn("payload_links", F.explode("payload.links"))
    .withColumn("payload_links_active", F.col("payload_links.active").cast("boolean"))
    .withColumn("payload_links_label", F.col("payload_links.label").cast("string"))
    .withColumn("payload_links_url", F.col("payload_links.url").cast("string"))
    .withColumn("payload_next_page_url", F.col("payload.next_page_url").cast("string"))
    .withColumn("payload_path", F.col("payload.path").cast("string"))
    .withColumn("payload_per_page", F.col("payload.per_page").cast("bigint"))
    .withColumn("payload_prev_page_url", F.col("payload.prev_page_url").cast("string"))
    .withColumn("payload_to", F.col("payload.to").cast("bigint"))
    .withColumn("payload_total", F.col("payload.total").cast("bigint"))
    .select(
        [
            "id",
            "ts",
            "payload_current_page",
            "payload_data",
            "payload_data_fact",
            "payload_data_length",
            "payload_first_page_url",
            "payload_from",
            "payload_last_page",
            "payload_last_page_url",
            "payload_links",
            "payload_links_active",
            "payload_links_label",
            "payload_links_url",
            "payload_next_page_url",
            "payload_path",
            "payload_per_page",
            "payload_prev_page_url",
            "payload_to",
            "payload_total",
        ]
    )
)

flattened_df.show()

+---+--------------------+--------------------+--------------------+--------------------+-------------------+----------------------+------------+-----------------+---------------------+--------------------+--------------------+-------------------+--------------------+---------------------+--------------------+----------------+---------------------+----------+-------------+
| id|                  ts|payload_current_page|        payload_data|   payload_data_fact|payload_data_length|payload_first_page_url|payload_from|payload_last_page|payload_last_page_url|       payload_links|payload_links_active|payload_links_label|   payload_links_url|payload_next_page_url|        payload_path|payload_per_page|payload_prev_page_url|payload_to|payload_total|
+---+--------------------+--------------------+--------------------+--------------------+-------------------+----------------------+------------+-----------------+---------------------+--------------------+--------------------+-------------------+-

23/08/03 23:33:25 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
